In [1]:
from tqdm import tqdm
import numpy as np
import torchaudio
import torch
import os
import wespeaker
import pandas as pd
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import os
import subprocess
import sys

In [2]:
os.chdir("..")

In [3]:
from src.utils import *

In [22]:
dataset_path = os.path.join(    
    "vox1_test_wav_bq_noise",
    "AirConditioner",
    "vox1_test_wav_snr_15_AirConditioner")
test_dir = os.path.join("data", dataset_path)
df = scan_directory_voxceleb1(test_dir)
df

,path,person_id,utterance_env,utterance_filename
0,data\vox1_test_wav_bq_noise\AirConditioner\vox...,id10270,5r0dWxy17C8,00001.wav
1,data\vox1_test_wav_bq_noise\AirConditioner\vox...,id10270,5r0dWxy17C8,00002.wav
2,data\vox1_test_wav_bq_noise\AirConditioner\vox...,id10270,5r0dWxy17C8,00003.wav
3,data\vox1_test_wav_bq_noise\AirConditioner\vox...,id10270,5r0dWxy17C8,00004.wav
4,data\vox1_test_wav_bq_noise\AirConditioner\vox...,id10270,5r0dWxy17C8,00005.wav
...,...,...,...,...
4869,data\vox1_test_wav_bq_noise\AirConditioner\vox...,id10309,_z_BR0ERa9g,00006.wav
4870,data\vox1_test_wav_bq_noise\AirConditioner\vox...,id10309,_z_BR0ERa9g,00007.wav
4871,data\vox1_test_wav_bq_noise\AirConditioner\vox...,id10309,_z_BR0ERa9g,00008.wav
4872,data\vox1_test_wav_bq_noise\AirConditioner\vox...,id10309,_z_BR0ERa9g,00009.wav


In [23]:
from pathlib import Path
from typing import Union
from src.utilities.prep_audio import get_audio_tensor

# for each row in df, extract audio tensor
output_dir = os.path.join(
    "data",
    "noisy_bg",
    dataset_path,
)
os.makedirs(output_dir, exist_ok=True)
for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    try:
        audio_tensor = get_audio_tensor(row["path"])
    except Exception as e:
        print(f"Error reading file {row['path']}: {e}")
        continue
    for i, segment in enumerate(audio_tensor):
        segment_name = "_seg_" + str(i)
        os.makedirs(output_dir + "/" + row["person_id"], exist_ok=True)
        segment_path = output_dir + "/" + row["person_id"] + "/" + row["utterance_filename"].replace(".wav","") + segment_name + ".wav"
        torchaudio.save(segment_path, segment, 16000)

100%|██████████| 4874/4874 [01:54<00:00, 42.73it/s]


In [25]:
def process_audio_segments(df, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    for index, row in tqdm(df.iterrows(), total=df.shape[0]):
        try:
            audio_tensor = get_audio_tensor(row["path"])
        except Exception as e:
            print(f"Error reading file {row['path']}: {e}")
            continue
        for i, segment in enumerate(audio_tensor):
            segment_name = "_seg_" + str(i)
            os.makedirs(output_dir + "/" + row["person_id"], exist_ok=True)
            segment_path = (
                output_dir
                + "/"
                + row["person_id"]
                + "/"
                + row["utterance_filename"].replace(".wav", "")
                + segment_name
                + ".wav"
            )
            torchaudio.save(segment_path, segment, 16000)



In [26]:
dataset_path = os.path.join(
    "vox1_test_wav_bq_noise", "AirConditioner", "vox1_test_wav_snr_20_AirConditioner"
)
test_dir = os.path.join("data", dataset_path)
df = scan_directory_voxceleb1(test_dir)
output_dir = os.path.join("data", "noisy_bg", dataset_path)

process_audio_segments(df, output_dir)

100%|██████████| 4874/4874 [01:57<00:00, 41.63it/s]


In [28]:
snr_levels = [15]
noise_types = [
    "Babble",
    "Neighbor",
]

for snr in snr_levels:
    for noise_type in noise_types:
        dataset_path = os.path.join(
            "vox1_test_wav_bq_noise",
            noise_type,
            f"vox1_test_wav_snr_{snr}_{noise_type}",
        )
        test_dir = os.path.join("data", dataset_path)
        df = scan_directory_voxceleb1(test_dir)
        output_dir = os.path.join("data", "noisy_bg", dataset_path)

        process_audio_segments(df, output_dir)

100%|██████████| 4874/4874 [02:02<00:00, 39.66it/s]


In [8]:
# check that the segments were created 

from src.utils import scan_directory_voxceleb2


segments = scan_directory_voxceleb2(output_dir)

In [9]:
segments

,path,person_id,utterance_filename
0,data\noisy_bg\vox1_test_wav_bq_noise\AirCondit...,id10270,00001_seg_0.wav
1,data\noisy_bg\vox1_test_wav_bq_noise\AirCondit...,id10270,00001_seg_1.wav
2,data\noisy_bg\vox1_test_wav_bq_noise\AirCondit...,id10270,00001_seg_2.wav
3,data\noisy_bg\vox1_test_wav_bq_noise\AirCondit...,id10270,00001_seg_3.wav
4,data\noisy_bg\vox1_test_wav_bq_noise\AirCondit...,id10270,00001_seg_4.wav
...,...,...,...
9114,data\noisy_bg\vox1_test_wav_bq_noise\AirCondit...,id10309,00016_seg_2.wav
9115,data\noisy_bg\vox1_test_wav_bq_noise\AirCondit...,id10309,00017_seg_0.wav
9116,data\noisy_bg\vox1_test_wav_bq_noise\AirCondit...,id10309,00018_seg_0.wav
9117,data\noisy_bg\vox1_test_wav_bq_noise\AirCondit...,id10309,00019_seg_0.wav


In [10]:
# Add a column with duration in samples
durations = []

for index, row in tqdm(segments.iterrows(), total=segments.shape[0]):
    try:
        info = torchaudio.info(row["path"])
        durations.append(info.num_frames)
    except Exception as e:
        print(f"Error reading file {row['path']}: {e}")
        durations.append(None)

segments["duration_samples"] = durations
unique_durations = segments["duration_samples"].unique()
print(unique_durations)

100%|██████████| 9119/9119 [00:53<00:00, 170.95it/s]

[64000]
